## Importando bibliotecas 

In [1]:
#Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import math
import config
#import matplotlib.pyplot as plt

from scipy.io.arff import loadarff

from sklearn.multioutput import MultiOutputRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold

#Importando as bibliotecas referentes ao XGBoost
import xgboost as xgb
from xgboost import XGBRegressor

#Importando as bibliotecas referentes ao ANN
import tensorflow as tf
from tensorflow import keras
from keras import layers , models
from keras.layers import Flatten, Dense, Dropout, Input
from keras.models import Sequential


## A fazer

RandomForest Global \
XGBoost

## Duvidas

É preciso normalizar os dados?? 
Se o ANN ja esta fazendo validation_split e epochs, precisa do kfold(computacionalmente intenso)



## Funções

### Modelos

In [3]:
#DecisionTree
def model_dtree(x_train,y_train,x_test,booster,eta,max_depth,colsample_bytree,subsample,min_child_weight,reg_lambda,num_boost,my_num):
    bst = DecisionTreeRegressor().fit(x_train, y_train)
    predict = bst.predict(x_test)
    return predict

#RandomForestRegressor
def model_randomForest(x_train,y_train,x_test,my_num):
    #booster,eta,max_depth,colsample_bytree,subsample,min_child_weight,reg_lambda,num_boost):
    bst = RandomForestRegressor(n_estimators = my_num,n_jobs = -1).fit(x_train,y_train)
    predict = bst.predict(x_test)
    return predict

#SVR
def model_svr(x_train,y_train,x_test,ker,gam,c,epsi,shrink,verb,max_it):
    bst = SVR(kernel=ker, gamma=gam, C=c, epsilon=epsi, shrinking=shrink, verbose=verb, max_iter=max_it).fit(x_train,y_train)
    predict = bst.predict(x_test)
    return predict

def model_multi_dtree(x_train, y_train, x_test):
    model = DecisionTreeRegressor()
    bst = MultiOutputRegressor(model).fit(x_train, y_train)
    predict = bst.predict(x_test)
    return predict

def model_ann_global(x_train, y_train, x_test):
    model = Sequential()
    model.add(Input(shape = (x_train.shape[1])))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1], activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=0, validation_split=0.2)
    
    predict = model.predict(x_test)
    return predict

def model_ann_local(x_train, y_train, x_test):
    predict_list = []
    for i in range(y_train.shape[1]):
        model = Sequential()
        model.add(Input(shape = x_train.shape[1]))
        model.add(Dense(x_train.shape[1]//2, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(x_train.shape[1]//6, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(x_train.shape[1]//18, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation='linear'))

        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        model.fit(x_train, y_train[:,i], epochs=10, batch_size=32, verbose=0, validation_split=0.2)

        predict = model.predict(x_test)
        predict_list.append(predict)
    
    predictions = np.concatenate(predict_list, axis=1)

    return predictions
    

### Cálculo de erro 

In [4]:
def computer_RRMSE_list(real_test,result_p,real_train_mean):
    """RRMSE: Relative Root Mean Square Error
        input: real_test: real test data
              result_p: predicted data
              real_train_mean: mean of real train data
        output: RRMSE list """

    _list = []
    for i in range(result_p.shape[1]):
        fenzi = 0
        fenmu = 0
        for j in range(result_p.shape[0]):
            fenzi += (result_p[j,i] - real_test[j,i])**2
            fenmu += (real_train_mean[i] - real_test[j,i])**2
        _list.append(math.sqrt(fenzi/fenmu))    
    return _list

## Implementação

In [36]:
path = "../TCC/mtr-datasets/"

#text_file = input("dataset: ")
text_file = 'edm'
data, meta = loadarff(path + text_file + ".arff")
data = pd.DataFrame(data)

data.reset_index(inplace=True)
data.replace('?', np.nan, inplace=True)
data.replace('     ?', np.nan, inplace=True)
data = data.applymap(float)

if config.all_config[text_file].get('sample_random') == True:
    data = data.sample(frac=1,random_state = config.all_config[text_file].get('sample_random_num')).reset_index(drop=True)
data = data.fillna(pd.Series(np.nanmean(data,axis=0),index=data.columns))
label = data.iloc[:,-config.all_config[text_file].get('targets_num'):].values
data = data.iloc[:,:-config.all_config[text_file].get('targets_num')].values

(154, 19)


#### MultiOutput Dtree

In [47]:
# Teste Multi-dtree
loss_list = [] 
kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'), shuffle=config.all_config[text_file].get('kfold_random'))
for train_index , test_index in kf.split(data):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    result_p_train = model_multi_dtree(x_train, y_train, x_test)
    loss_list_RRMSE = computer_RRMSE_list(y_test,result_p_train,np.mean(y_train,axis=0))

print("RRMSE: ", loss_list_RRMSE)
print("RRMSE mean: ", np.mean(loss_list_RRMSE))

RRMSE:  [0.8945120735947261, 0.7071067811865476]
RRMSE mean:  0.8008094273906368


#### Random Forest Local 

In [40]:
loss_list_RRMSE = []

kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'),
            shuffle=config.all_config[text_file].get('kfold_random'),
            random_state=config.all_config[text_file].get('kfold_random_num'))

for train_index, test_index in kf.split(data):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]

    result_p_train = []
    
    for i in range(label.shape[1]):

        predicted_test = None 

        if text_file in ['sf1', 'sf2']:
            predicted_test = model_svr(x_train, y_train[:,i], x_test,
                                    config.all_config[text_file].get('svr1_kernel'),
                                    config.all_config[text_file].get('svr1_gamma'),
                                    config.all_config[text_file].get('svr1_C'),
                                    config.all_config[text_file].get('svr1_epsilon'),
                                    config.all_config[text_file].get('svr1_shrinking'),
                                    config.all_config[text_file].get('svr1_verbose'),
                                    config.all_config[text_file].get('svr1_max_iter'))
        else:
            predicted_test = model_randomForest(x_train, y_train[:,i], x_test, my_num=100)
            
        result_p_train.append(predicted_test)

    result_p_train = pd.DataFrame(result_p_train).T.values
    real_train_mean = y_train.mean(axis=0)    
    loss_list_RRMSE.append(computer_RRMSE_list(y_test,result_p_test,real_train_mean))


loss_list_RRMSE_np = np.array(loss_list_RRMSE)
print('RRMSE: ', loss_list_RRMSE_np)
print('RRMSE mean: ',loss_list_RRMSE_np.mean(axis=0))


RRMSE:  [[0.57379056 0.59367751]
 [0.5142302  0.55673731]
 [0.65645042 0.45810442]
 [0.56435736 0.63305913]
 [0.16542698 0.66258362]
 [0.87163915 0.58206674]
 [1.02031143 0.67596228]
 [0.3191708  0.49182511]
 [0.61531383 1.03279808]
 [0.49860523 0.72015045]]
RRMSE mean:  [0.5799296  0.64069646]


#### ANN Global 

In [42]:
loss_list_RRMSE = []

kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'),
            shuffle=config.all_config[text_file].get('kfold_random'),
            random_state=config.all_config[text_file].get('kfold_random_num'))

for train_index, test_index in kf.split(data):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]

    result_p_train = model_ann_global(x_train, y_train, x_test)  # Call model_ann_global once
    
    real_train_mean = y_train.mean(axis=0)    
    loss_list_RRMSE.append(computer_RRMSE_list(y_test, result_p_train, real_train_mean))

loss_list_RRMSE_np = np.array(loss_list_RRMSE)
print('RRMSE: ', loss_list_RRMSE_np)
print('RRMSE mean: ', loss_list_RRMSE_np.mean(axis=0))

1/1 [==============================] - 0s 41ms/step
RRMSE:  [[2.40731774 2.7391447 ]
 [4.0553366  1.11888183]
 [1.19307816 1.62058992]
 [4.4477298  1.2102089 ]
 [8.04938791 1.32459004]
 [2.65017405 1.38400914]
 [3.61227642 1.6584659 ]
 [5.15111159 1.50364246]
 [1.45767028 1.43351679]
 [1.81259779 1.99189972]]
RRMSE mean:  [3.48366803 1.59849494]


#### ANN Local

In [41]:
loss_list_RRMSE = []

kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'),
            shuffle=config.all_config[text_file].get('kfold_random'),
            random_state=config.all_config[text_file].get('kfold_random_num'))

for train_index, test_index in kf.split(data):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]

    result_p_train = model_ann_local(x_train, y_train, x_test)
   
    real_train_mean = y_train.mean(axis=0)    

    result_p_train = pd.DataFrame(result_p_train).values
    loss_list_RRMSE.append(computer_RRMSE_list(y_test, result_p_train, real_train_mean))

    
loss_list_RRMSE_np = np.array(loss_list_RRMSE)
print('RRMSE: ', loss_list_RRMSE_np)
print('RRMSE mean: ',loss_list_RRMSE_np.mean(axis=0))

1/1 [==============================] - 0s 47ms/step
RRMSE:  [[0.98918645 0.98828124]
 [0.99676859 0.98976319]
 [1.0481249  0.97595998]
 [0.97642306 0.99505678]
 [0.86068254 0.99804417]
 [1.04376054 1.00109515]
 [0.99634256 1.00144899]
 [1.04099167 0.99081032]
 [1.04024255 0.99975905]
 [1.01737861 0.99591707]]
RRMSE mean:  [1.00099015 0.99361359]


In [48]:
# Ta errado ainda, 
# A funcao model_ann_local ta iterando sobre as colunas
# e nesse código ta iterando de novo sobre as colunas

loss_list_RRMSE = []

kf = KFold(n_splits=config.all_config[text_file].get('paper_kFold'),
            shuffle=config.all_config[text_file].get('kfold_random'),
            random_state=config.all_config[text_file].get('kfold_random_num'))

for train_index, test_index in kf.split(data):
    x_train, x_test = data[train_index], data[test_index]
    y_train, y_test = label[train_index], label[test_index]

    result_p_test = []

    for i in range(label.shape[1]):
        
        predicted_test = None 

        if text_file in ['sf1', 'sf2']:
            predicted_test = model_svr(x_train, y_train[:,i], x_test, y_test[:,i],
                                    config.all_config[text_file].get('svr1_kernel'),
                                    config.all_config[text_file].get('svr1_gamma'),
                                    config.all_config[text_file].get('svr1_C'),
                                    config.all_config[text_file].get('svr1_epsilon'),
                                    config.all_config[text_file].get('svr1_shrinking'),
                                    config.all_config[text_file].get('svr1_verbose'),
                                    config.all_config[text_file].get('svr1_max_iter'))
        else:
            predicted_test = model_ann_local(x_train, y_train, x_test)
        
        result_p_test.append(predicted_test)

    result_p_test = pd.DataFrame(result_p_test).T.values
    real_train_mean = y_train.mean(axis=0)    
    loss_list_RRMSE.append(computer_RRMSE_list(y_test,result_p_test,real_train_mean))


loss_list_RRMSE_np = np.array(loss_list_RRMSE)
print('RRMSE: ',np.mean(loss_list_RRMSE_np))

2/2 [==============================] - 0s 1ms/step


ValueError: Must pass 2-d input. shape=(6, 34, 6)